## Классификация комментариев

### 1. Подготовка данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import warnings
from tqdm import notebook
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

Прочитаем файл и сохраним его в переменной *comments*.

In [2]:
comments = pd.read_csv('/datasets/toxic_comments.csv')

Посмотрим таблицу и общую информацию о ней.

In [3]:
display(comments.sample(5))
comments.info()

,text,toxic
121611,"Back Door Key \n\nA colleague of mine, who is ...",0
134727,REDIRECT Talk:Lega Basket A,0
42260,Natalie Started as a Correspondent and Sub anc...,0
82596,Military history WikiProject Newsletter - Issu...,0
150578,"""\n\n Thank you for the Barnstar \n\nDear Dani...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Датафрейм содержит 159571 строк и столбцы с комментариями на английском языке и бинарными оценками их токсичности. Очистим комментарии от лишних символов, оставив только буквы английского алфавита. Сохраним результат в столбце *cleared*.

In [4]:
warnings.filterwarnings('ignore')

def clear(comment):
    cleared = re.sub(r'[^a-zA-Z]', ' ', comment)    
    return cleared

notebook.tqdm.pandas()

comments['cleared'] = comments['text'].progress_apply(clear)

Проведём лемматизацию очищенных комментариев с удалением стоп-слов. Сохраним результат в столбце *lemmatized*.

In [ ]:
nlp = spacy.load('en_core_web_sm')

def lemmatize(comment):  
    doc = nlp(comment)
    lemmatized = ' '.join([token.lemma_ for token in doc if (token.is_stop==False) & \
                                                            (token.shape_ not in ['x', 'X']) & \
                                                            (token.pos_!='SPACE')])
    return lemmatized

comments['lemmatized'] = comments['cleared'].progress_apply(lemmatize)

Проверим успешность очистки и лемматизации комментариев.

In [ ]:
comments[['text', 'cleared', 'lemmatized', 'toxic']].sample(5)

После удаления стоп-слов в данных могли появиться пропуски. Удалим строки, которые их содержат.

In [7]:
comments = comments.dropna().reset_index(drop=True)

### 2. Обучение модели

Обучим модели на лемматизированных и не лемматизированных данных, а также с использованием и без использования биграмм. Для этого создадим словарь *d* с возможными комбинациями параметров.

In [8]:
d = {0: ['cleared', 1],
     1: ['cleared', 2],
     2: ['lemmatized', 1],
     3: ['lemmatized', 2]}

Выделим целевой признак и разделим датасет на обучающую и тестовую выборки.

In [9]:
features_train = pd.DataFrame()
features_test = pd.DataFrame()

features_train['cleared'], features_test['cleared'], \
features_train['lemmatized'], features_test['lemmatized'], \
target_train, target_test = train_test_split(comments['cleared'],
                                             comments['lemmatized'],
                                             comments['toxic'],
                                             test_size=0.2,
                                             random_state=1,
                                             stratify=comments['toxic'])

На основе словаря *d* обучим по сеткам с различными гиперпараметрами модели логистической регрессии и градиентного бустинга *LightGBM*. Определим гиперпараметры с наилучшими метриками F1 по итогам кросс-валидации.

In [10]:
lr_results = {}
lgbm_results = {}

for i in notebook.tqdm(d):
    print('Лемматизация: {:<2} Биграммы: {} \n'.format(('✔' if d[i][0] == 'lemmatized' else '✖'),
                                                       ('✔' if d[i][1] == 2 else '✖')))
    # Векторизация
    count_tf_idf_train = TfidfVectorizer(ngram_range=(1, d[i][1]))
    tf_idf_train = count_tf_idf_train.fit_transform(features_train[d[i][0]])
    tf_idf_test = count_tf_idf_train.transform(features_test[d[i][0]])
    
    # Логистическая регрессия
    lr = LogisticRegression(random_state=1, class_weight='balanced', solver='liblinear')
    lr_parameters = {'C': np.logspace(-2, 2, 5), 
                     'penalty': ['l1', 'l2']}
    
    lr_grid = GridSearchCV(lr, lr_parameters, cv=4, n_jobs=-1, scoring='f1')
    lr_grid.fit(tf_idf_train, target_train)
    
    lr_results[i] = pd.DataFrame(lr_grid.cv_results_)
    print('Логистическая регрессия')
    print('Лучшие гиперпараметры: ', (lr_grid.best_params_))
    print('F1 = {:.2f} \n'.format(lr_grid.best_score_))
    
    # LightGBM
    lgbm = LGBMClassifier(n_estimators=150, class_weight='balanced')
    lgbm_parameters = {'learning_rate': np.arange(0.1, 0.31, 0.1),
                       'num_leaves': [7, 15]}
    lgbm_grid = GridSearchCV(lgbm, lgbm_parameters, cv=4, n_jobs=-1, scoring='f1')
    lgbm_grid.fit(tf_idf_train, target_train)
    
    lgbm_results[i] = pd.DataFrame(lgbm_grid.cv_results_)
    print('LightGBM')
    print('Лучшие гиперпараметры: ', lgbm_grid.best_params_)
    print('F1 = {:.2f}'.format(lgbm_grid.best_score_))
    
    if i < len(d)-1:
        print('—' * 120)

Лемматизация: ✖  Биграммы: ✖ 

Логистическая регрессия
Лучшие гиперпараметры:  {'C': 10.0, 'penalty': 'l2'}
F1 = 0.77 

LightGBM
Лучшие гиперпараметры:  {'learning_rate': 0.30000000000000004, 'num_leaves': 15}
F1 = 0.73
————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Лемматизация: ✖  Биграммы: ✔ 

Логистическая регрессия
Лучшие гиперпараметры:  {'C': 10.0, 'penalty': 'l1'}
F1 = 0.78 

LightGBM
Лучшие гиперпараметры:  {'learning_rate': 0.30000000000000004, 'num_leaves': 15}
F1 = 0.73
————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Лемматизация: ✔  Биграммы: ✖ 

Логистическая регрессия
Лучшие гиперпараметры:  {'C': 10.0, 'penalty': 'l2'}
F1 = 0.76 

LightGBM
Лучшие гиперпараметры:  {'learning_rate': 0.30000000000000004, 'num_leaves': 15}
F1 = 0.75
——————————————————————————————————————————————————————————————————————————————————————————————————

Все модели показали близкие результаты, однако качество предсказаний логистической регрессии немного выше, чем у *LightGBM*. Наилучшее значение метрики F1 (0.78) показали модели логистической регрессии, обученные на данных с использованием биграмм. Исследуем их результаты более подробно.

In [11]:
for i in [1, 3]:
    print('\n Лемматизация: {}'.format(('✔' if d[i][0] == 'lemmatized' else '✖')))
    display(lr_results[i].iloc[:, [4, 5, 0, 11]].sort_values('mean_test_score', ascending=False).head(5))


 Лемматизация: ✖


,param_C,param_penalty,mean_fit_time,mean_test_score
6,10,l1,48.191428,0.784939
7,10,l2,47.099354,0.782292
8,100,l1,27.033905,0.781967
9,100,l2,64.508006,0.781428
4,1,l1,22.455938,0.752586



 Лемматизация: ✔


,param_C,param_penalty,mean_fit_time,mean_test_score
7,10,l2,29.272216,0.778280
6,10,l1,1830.626293,0.777992
8,100,l1,33.939060,0.775412
9,100,l2,45.879880,0.774798
5,1,l2,16.684891,0.760014


Из полученных таблиц видно, что качество моделей, обученных на нелемматизированных данных, немного выше. При этом время их обучения имеет тот же порядок (за исключением аномально долгого обучения одной из моделей на лемматизированных данных), а с учётом отсутствия временных затрат на лемматизацию они работают даже быстрее. Исходя из этого, в качестве финальной модели выберем модель логистической регрессии с l1-регуляризацией и обратным коэффициентом регуляризации, равным 10, F1 которой на кросс-валидации составила 0.7849. Обучим её на всей тренировочной выборке без лемматизации, а также с использованием униграмм и биграмм. Проверим качество предсказаний на тестовой выборке.

In [12]:
# Векторизация
count_tf_idf_train = TfidfVectorizer(ngram_range=(1, 2))
tf_idf_train = count_tf_idf_train.fit_transform(features_train['cleared'])
tf_idf_test = count_tf_idf_train.transform(features_test['cleared'])

# Логистическая регрессия
lr = LogisticRegression(random_state=1, class_weight='balanced', solver='liblinear', C=10, penalty='l1')
lr.fit(tf_idf_train, target_train)
predicted = lr.predict(tf_idf_test)
print('F1 = {:.4f}'.format(f1_score(target_test, predicted)))

F1 = 0.7809


Таким образом, итоговая метрика F1 на тестовой выборке составила 0.7809, что соответствует целевому показателю. Также можно отметить, что качество почти не изменилось по сравнению со значением, полученным на кросс-валидации. Это говорит о том, что удалось избежать переобучения модели.

### 3. Вывод

В ходе выполнения проекта проведена подготовка данных — тексты комментариев очищены от лишних символов и проведена лемматизция с удалением стоп-слов; на данных с лемматизацией и без неё, с использованием и без использования биграмм в дополнение к униграммам обучены модели логистической регрессии и градиентного бустинга на основе библиотеки *LightGBM* c различными гиперпараметрами. Установлено, что модели градиентного бустинга немного уступают в качестве моделям логистической регрессии. Наилучшего результата удалось достичь при обучении на нелемматизированных данных с использованием униграмм и биграмм. В качестве финальной выбрана модель логистической регрессии со следующими гиперпараметрами:
* class_weight='balanced';
* solver='liblinear';
* C=10;
* penalty='l1'.

Проведена проверка данной модели на тестовой выборке. Итоговая метрика F1 оказалась равна 0.7809.